## Przykładowe funkcje i parametry i wykorzystywane paczki

In [ ]:
f(x) = sin.(x) .* sqrt.(x)
ReLu(x) = max.(zero.(x),x)
σ(x) = one.(x)./(one.(x).+exp.(-x))
softmax(x) = exp.(x) ./ sum(exp.(x))

x1 = rand(Float64, 2);
x2 = rand(Float64, 100);

using BenchmarkTools
using ForwardDiff
using ReverseDiff

# Różniczkowanie w przód

In [ ]:
struct Dual{T <:Number} <:Number
    v::T
    dv::T
end

In [ ]:
import Base: +, -, *, /
     -(x::Dual) = Dual(-x.v, -x.dv)
     +(x::Dual, y::Dual) = Dual( x.v + y.v, x.dv + y.dv)
     -(x::Dual, y::Dual) = Dual( x.v - y.v, x.dv - y.dv)
     *(x::Dual, y::Dual) = Dual( x.v * y.v, x.dv * y.v + x.v * y.dv)
     /(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)
 
import Base: abs, sin, cos, tan, exp, sqrt, isless
    abs(x::Dual) = Dual(abs(x.v),sign(x.v)*x.dv)
    sin(x::Dual) = Dual(sin(x.v), cos(x.v)*x.dv)
    cos(x::Dual) = Dual(cos(x.v),-sin(x.v)*x.dv)
    tan(x::Dual) = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
    exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
    sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
    isless(x::Dual, y::Dual) = x.v < y.v;

In [ ]:
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

In [ ]:
import Base: convert, promote_rule
convert(::Type{Dual{T}}, x::Dual) where T =
 Dual(convert(T, x.v), convert(T, x.dv))
convert(::Type{Dual{T}}, x::Number) where T =
 Dual(convert(T, x), zero(T))
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} =
 Dual{promote_type(T,R)}

## Funkcja obliczająca macierz Jacobiego 

In [ ]:
J = function jacobian(f, args::Vector{T}) where {T <:Number} # przyjmuje jako argumenty funkcje oraz wektor argumentów
    jacobian_columns = Matrix{T}[]
    for i=1:length(args)
        x = Dual{T}[] # tworzy nowy wektor x dla liczb dualnych
        for j=1:length(args)
            seed = (i == j)
            # dodaje do wektora x liczbe dualną z zarodkiem lub bez w zależności od wartości seed 
            push!(x, seed ? Dual(args[j], one(args[j])) : Dual(args[j],zero(args[j])))
            
            #println("x to = ",x)
            #println("iteracja i = ",i," j = ",j)
        end
        #println(f(x))
        column = partials.([f(x)...]) # tworzy wektor pochodnych        
        push!(jacobian_columns, column[:,:]) # i dodaje do macierzy wynikowej
        
    end
    hcat(jacobian_columns...) # zamienia w macierz układając zbiór wektorów w kolumny 
end

## Test macierzy Jacobiego dla różniczkowania w przód

In [ ]:
println("Test f dla wektora x1")
@btime begin
J(f, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(f, x1)   
end

println(J(f, x1))

In [ ]:
println("Test f dla wektora x2")
@btime begin
J(f, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(f, x2)   
end

In [ ]:
println("Test ReLu dla wektora x1")
@btime begin
J(ReLu, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(ReLu, x1)
end

In [ ]:
println("Test ReLu dla wektora x2")
@btime begin
J(ReLu, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(ReLu, x2)   
end

In [ ]:
println("Test σ dla wektora x1")
@btime begin
J(σ, x1)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(σ, x1)   
end

In [ ]:
println("Test σ dla wektora x2")
@btime begin
J(σ, x2)
end

println("Sprawdzenie ForwardDiff")
@btime begin
ForwardDiff.jacobian(σ, x2)   
end

# Różniczkowanie w tył

In [ ]:
abstract type AbstractNode end

mutable struct Variable{T} <: AbstractNode
    value::T
    grad::T
    
    Variable(val::T) where T = new{T}(val, zero(val))
    Variable(val::T, grad::T) where T = new{T}(val, grad)
end

struct Node{FT <: Function, ArgsT} <: AbstractNode
    f::FT # funkcja wykonywana w node
    args::ArgsT # argumenty funkcji
end

mutable struct CachedNode{NT <: AbstractNode, OutT} <: AbstractNode
    node::NT # Node
    output::OutT # wynik operacji na zadanych argumentach w computable Node
end

function CachedNode(f, args...)
    println(f)
    node = Node(f, args)
    output = forward(node)
    CachedNode(node, output)
end

In [ ]:
forward(cached::CachedNode) = cached.output = forward(cached.node)
forward(node::Node) = forward(node.f, map(forward, node.args)...) # dokonujemy mapowania funkcją forward aby dobrac się do value zmiennych variable
forward(f::Function, args...) = f(args...) # finalne wykonanie operacji na zadanych argumentach i obliczenie wyniku
forward(var::Variable) = var.value # wykorzystywane do mapowania

In [ ]:
value(x) = x
value(x::Variable) = x.value
value(x::CachedNode) = value(x.output)

In [ ]:
import Base: convert, promote_rule
convert(::Type{Variable{T}}, x::Variable) where T =
 Variable(convert(T, x.value), convert(T, x.grad))
convert(::Type{Variable{T}}, x::Number) where T =
 Variable(convert(T, x), zero(T))
promote_rule(::Type{Variable{T}}, ::Type{R}) where {T,R} =
 Variable{promote_type(T,R)}

In [ ]:
# rejestracja operacji z wyrażenia wejściowego
register(f, args...) = CachedNode(f, args...)

import Base: +, -, *, /
    -(x::AbstractNode) = register(-, x)
    +(x::AbstractNode, y::AbstractNode) = register(+, x, y)
    -(x::AbstractNode, y::AbstractNode) = register(-, x, y)
    *(x::AbstractNode, y::AbstractNode) = register(*, x, y)
    #*(x::AbstractNode, y::Number) = register(*, x, Variable(y))
    /(x::AbstractNode, y::AbstractNode) = register(/, x, y)
import Base: abs, sin, cos, tan, exp, sqrt, zero, one, max, min, length, iterate, sum
    abs(x::AbstractNode) = register(abs, x)
    sin(x::AbstractNode) = register(sin, x)
    cos(x::AbstractNode) = register(cos, x)
    tan(x::AbstractNode) = register(tan, x)
    exp(x::AbstractNode) = register(exp, x)
    sqrt(x::AbstractNode) = register(sqrt, x)
    zero(x::AbstractNode) = register(zero, x)
    one(x::AbstractNode) = register(one, x)
    max(x::AbstractNode, y::AbstractNode) = register(max, x, y)
    min(x::AbstractNode, y::AbstractNode) = register(min, x, y)
    length(x::AbstractNode) = length(value(x))
    iterate(x::AbstractNode) = iterate_forward(iterate(value(x)), x)
    iterate(x::AbstractNode, st) = iterate_forward(iterate(value(x), st), x, st)
    #sum(x::Value{<:AbstractArray}; dims=:) = register(Base.sum, x; dims=dims)

iterate_forward(out::Nothing, x::AbstractNode) = nothing
iterate_forward(out::Nothing, x::AbstractNode, st) = nothing

function iterate_forward(out, x::AbstractNode, st)
   node = Node(Base.iterate, (x, st))
   v, st = out
   CachedNode(node, v), st
end

function iterate_forward(out, x::AbstractNode)
    node = Node(iterate, (x, ))
    v, st = out
    CachedNode(node, v), st
end

In [ ]:
function backward(cached::CachedNode, grad)
    grad_inputs = gradient(cached, grad)
    for (each, each_grad) in zip(cached.node.args, grad_inputs)
        backward(each, each_grad)
    end
end

function backward(var::Variable, grad)
    var.grad += grad
end

gradient(cached::CachedNode, grad) =
    gradient(cached.node.f, grad, cached.output, map(value, cached.node.args)...)

gradient(::typeof(-), grad, output, x) = (-grad, )
gradient(::typeof(+), grad, output, x, y) = (grad, grad)
gradient(::typeof(-), grad, output, x, y) = (grad, -grad)
gradient(::typeof(*), grad, output, x, y) = (grad * y, grad * x) # (pochodna po x, pochodna po y)
gradient(::typeof(/), grad, output, x, y) = (grad * one(x)/y, grad * -(x / (y * y)))
gradient(::typeof(abs), grad, output, x) = (grad * x/abs(x), ) 
gradient(::typeof(sin), grad, output, x) = (grad * cos(x), ) # dodajemy ',' aby cały czas to był tuple
gradient(::typeof(cos), grad, output, x) = (grad * -sin(x), )
gradient(::typeof(tan), grad, output, x) = (grad * (tan(x)*tan(x)+1), )
gradient(::typeof(exp), grad, output, x) = (grad * exp(x), )
gradient(::typeof(sqrt), grad, output, x) = (grad * 1/(2*sqrt(x)), )
gradient(::typeof(min), grad, output, x, y) = (isless(x, y) ? grad * one(x) : grad * zero(x), 
                                          isless(x, y) ? grad * zero(y) : grad * one(y))
gradient(::typeof(max), grad, output, x, y) = (isless(x, y) ? grad * zero(x) : grad * one(x), 
                                          isless(x, y) ? grad * one(y) : grad * zero(y))
gradient(::typeof(zero), grad, output, x) = (grad * zero(x), ) # zero czy one to po prostu funkcje stałe
gradient(::typeof(one), grad, output, x) = (grad * zero(x), )
#gradient(::typeof(sum), grad, output, x::AbstractArray; dims) =
#    grad_sum(grad, x, dims)

#grad_sum(grad, x, dims::Colon) = (fill!(similar(x), grad), )

function gradient(::typeof(iterate), grad::Number, output, x::AbstractArray)
    out_grad = zero(x)
    out_grad[1] = grad
    (out_grad, )
end

# function gradient(::typeof(iterate), grad::Number, output, x::Number)
#     (zero(grad), )
# end

function gradient(::typeof(iterate), grad, output, x::AbstractArray, st)
    out_grad = zero(x)
    out_grad[st] = grad
    (out_grad, )
end

In [ ]:
function jacobian(cached::CachedNode)
    jjj = []
    println("In Jacobian")
    for each in cached.node.args
        #println("------")
        #println(each)
        x = jacobian(each)
        #println(x)
        push!(jjj, x)
        #println(jjj)
    end
    return jjj
end

function jacobian(var::Variable)
    #println("_________")
    #println(var.grad)
    return var.grad
end

In [ ]:
J2 = function jacobian2(f, args::Vector{T}) where {T <:Number} # przyjmuje jako argumenty funkcje oraz wektor argumentów
    x = map(x -> Variable(x), args)
    println(x)
    y = f(x)
    println("Before backward")
    backward.(y, 1.0)
    println(y)
    diagonal = jacobian.(y)
    size = length(diagonal)
    Jacobian = zeros(size, size)
    for i=1:size
        Jacobian[i,i] = getJacobianValue(diagonal[i])
    end
    #println("jacobian:")
    #println(Jacobian)
    return Jacobian
end

function getJacobianValue(val::Array{Any})
    return getJacobianValue(val[length(val)])
end

function getJacobianValue(val::Any)
    return val
end

## Testy macierzy Jacobiego dla różniczkowania w tył

In [ ]:
println("Test f dla wektora x1")
#@btime begin
J2(f, x1)
#end

println("Sprawdzenie ReverseDiff")
#@btime begin
y = ReverseDiff.jacobian(f, x1)
#end
println(y)
println(J2(f, x1))

In [ ]:
println("Test f dla wektora x2")
@btime begin
J2(f, x2)
end

println("Sprawdzenie ReverseDiff")
@btime begin
ReverseDiff.jacobian(f, x2)
end

In [ ]:
println("Test ReLu dla wektora x1")
@btime begin
J2(ReLu, x1)
end

println("Sprawdzenie ReverseDiff")
@btime begin
ReverseDiff.jacobian(ReLu, x1)
end

In [ ]:
println("Test ReLu dla wektora x2")
@btime begin
J2(ReLu, x2)
end

println("Sprawdzenie ReverseDiff")
@btime begin
ReverseDiff.jacobian(ReLu, x2)
end

In [ ]:
println("Test σ dla wektora x1")
#@btime begin
println(J2(σ, x1))
#end

println("Sprawdzenie ReverseDiff")
#@btime begin
ReverseDiff.jacobian(σ, x1)
#end

In [ ]:
println("Test σ dla wektora x2")
@btime begin
J2(σ, x2)
end

println("Sprawdzenie ReverseDiff")
@btime begin
ReverseDiff.jacobian(σ, x2)
end

In [ ]:
println("Test softmax dla wektora x1")
#@btime begin
println(J2(softmax, x1))
#end

println("Sprawdzenie ReverseDiff")
#@btime begin
ReverseDiff.jacobian(softmax, x1)
#end

Sieć neuronowa

In [ ]:
dense(w, n, m, v, f) = f.(reshape(w, n, m) * v)
mean_squared_loss(y, ŷ) = sum(0.5(y - ŷ).^2)
σ(x) = one.(x) ./ (one.(x) .+ exp.(-x))
linear(x) = x

Wh = randn(10,2)
Wo = randn(1,10)
dWh = similar(Wh)
dWo = similar(Wo)

x = [1.98;4.434]
y = [0.064]

function net(x, wh, wo, y)
    x̂ = dense(wh, 10, 2, x, σ)
    ŷ = dense(wo, 1, 10, x̂, linear)
    E = mean_squared_loss(y, ŷ)
end

dnet_Wh(x, wh, wo, y) = J2(w -> net(x, w, wo, y), wh);
dnet_Wo(x, wh, wo, y) = J2(w -> net(x, wh, w, y), wo);

epochs = 10
for i=1:epochs
    E = net(x, Wh[:], Wo[:], y)
    println(E)
    dWh[:] = dnet_Wh(x, Wh[:], Wo[:], y)
    dWo[:] = dnet_Wo(x, Wh[:], Wo[:], y)
    Wh -= 0.1dWh
    Wo -= 0.1dWo
end